## Importing Packages

In [1]:
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, ReLU, LSTM, GRU, SimpleRNN, Dense, TimeDistributed, Bidirectional, GaussianNoise 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential, model_from_json
import tensorflow as tf
import tensorflow_model_remediation.min_diff.losses.mmd_loss as MMD

import matplotlib.pyplot as plt
from itertools import chain
import pandas as pd
import numpy as np
import imageio
import os
import re

print(tf.__version__)

2.9.0


## Dataset Reading and Preprocessing

### Listing all the movable joints in the human skeleton

In [2]:
#Was done in the preprocessing in [1]
# Joints in H3.6M -- data has 32 joints, but only 17 that move; these are the indices.
H36M_NAMES = ['']*32
H36M_NAMES[0]  = 'Hip'
H36M_NAMES[1]  = 'RHip'
H36M_NAMES[2]  = 'RKnee'
H36M_NAMES[3]  = 'RFoot'
H36M_NAMES[6]  = 'LHip'
H36M_NAMES[7]  = 'LKnee'
H36M_NAMES[8]  = 'LFoot'
H36M_NAMES[12] = 'Spine'
H36M_NAMES[13] = 'Thorax'
H36M_NAMES[14] = 'Neck/Nose'
H36M_NAMES[15] = 'Head'
H36M_NAMES[17] = 'LShoulder'
H36M_NAMES[18] = 'LElbow'
H36M_NAMES[19] = 'LWrist'
H36M_NAMES[25] = 'RShoulder'
H36M_NAMES[26] = 'RElbow'
H36M_NAMES[27] = 'RWrist'

### A class to Read and Combine all the Dataset files

In [3]:
class Dataset_loading:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, 
                 total_classes = 17, datatype = 'float32', include_movable_joints = False, batch_size = 20, 
                 include_action_labels = True, return_action_labels = False):
        
        #Dataset Directory path
        self.dir_path = dir_path
        
        #Which Dimension file to include, possible values: 2 and 3
        self.include_dimension = include_dimension
        
        #Total frames in one Sample
        self.sample_size = sample_size
        
        #Default Datatype for all the samples
        self.datatype = datatype
        
        #Batch Size of the dataset for experimentation 
        self.batch_size = batch_size
        
        #Boolean value to indicate whether to include action class in each frames
        self.include_action_labels = include_action_labels
        
        #Whether to return action labels with data
        self.return_action_labels = return_action_labels
        
        #Activity classes to include
        self.classes = ['SittingDown', 'Walking', 'Directions', 'Discussion', 'Sitting', 'Phoning', 'Eating', 'Posing', 'Greeting', 'Smoking']
        
        #Total activity classes
        self.total_classes = len(self.classes)
        
        #Subject Folders names in the Dataset
        self.internal_folders = ['S1', 'S5','S6','S7','S8','S9','S11']

        #Boolean value indicating whether to include all joints or only the movable joints.
        self.include_movable_joints = include_movable_joints
        
        self.movable_joints = [0, 1, 2, 3, 6, 7, 8, 12, 13, 14, 15, 17, 18, 19, 25, 26, 27]
    
    def read_dataset(self):
        try:
            #Contains all the different activity vectors
            activity_vector = {}
            
            #Contains the overall dataset
            sampled_data = None
            sampled_labels = None
            
            #Based on dimensions, which folder to use for extracting the dataset files
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            
            #Checking if the dataset path is valid
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None

            #Iterating over all the subject folders
            for fld in self.internal_folders:
                #Iterating for each file in the specified folder
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    #Extracting the activity from the filename
                    activity = self.__extract_activity(file)
                    
                    if activity not in self.classes:
                        continue
                    
                    #Reading the CSV file using Pandas
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)

                    #Formulating the activity vector using one hot encoding
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    
                    #Sampling the dataset
                    grouped_sample, grouped_activity = self.__group_samples(data, self.sample_size, vector)
                    sampled_data = grouped_sample if sampled_data is None else np.append(sampled_data, grouped_sample, axis=0)
                    sampled_labels = grouped_activity if sampled_labels is None else np.append(sampled_labels, grouped_activity, axis=0)
            
            #Changing the Datatype
            sampled_data = sampled_data.astype(self.datatype)
            
            #To make the data divisible for batch size
            total_batches = sampled_data.shape[0]
            sampled_data = sampled_data[:total_batches - (total_batches % self.batch_size)]
            sampled_labels = sampled_labels[:total_batches - (total_batches % self.batch_size)]
            
            if self.return_action_labels:
                return sampled_data, sampled_labels
            
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            #Extracting the filename and excluding the extension
            name = os.path.splitext(filename)[0]
            
            #Substituting the empty string with characters other than english alphabets
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            #Checking if the dataset is a Pandas Dataframe
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            if self.include_movable_joints:
                joints = list(chain.from_iterable((jt*2, (jt*2)+1) for jt in self.movable_joints))
                dataset = dataset.iloc[: , joints].copy()

            #Appending activity class to each row in the dataset
            if self.include_action_labels:
                dataset = pd.concat([dataset, pd.DataFrame(np.tile(activity, (dataset.shape[0],1)))], axis=1)
            
            #Reshaping the dataset into sample batches
            total_samples = dataset.shape[0]//sample_size
            total_features = dataset.shape[1]
            grouped_rows = dataset.to_numpy()[:total_samples*self.sample_size].reshape((-1,self.sample_size, total_features))
            
            grouped_activity = np.tile(activity, (dataset.shape[0]//self.sample_size, 1))
            grouped_activity = grouped_activity[:total_samples*self.sample_size].reshape((-1, len(activity)))
            
            return grouped_rows, grouped_activity
        except Exception as e:
            print(e)

### A function to split Dataset into Features and Labels

In [4]:
def split_to_features_labels(dataset, input_sequance_size=10, total_features=64):
    """
    Function for splitting the data into features(with sequance size=iput_sequance_size)
    and labels which should be the remainder of the sample length 
    """
    assert input_sequance_size < dataset.shape[1], f"input sequence should be smaller than the total sample size"
    
    #Dividing the dataset into features and labels by splitting the Time Frame Dimension
    features = dataset[:, np.s_[0:input_sequance_size], :]
    labels = dataset[:,np.s_[input_sequance_size:], :total_features]
    
    return features, labels

### A function for downsampling the dataset on number of frames

In [5]:
def downsampling(sampled_data, downsample_technique = 'skip'):
    """
    The function used to down-sample the data using two different techniques. In Skip, one frame is skipped consecutively and
    in the mean technique, two frames are averaged consecutively.
    """
    
    assert downsample_technique in ['skip', 'mean'], f"Only Skip and Mean techniques are available"
    
    #Creating an empty variable to store Downsampled data when the technique is Mean
    samples_per_batch = int(sampled_data.shape[1] / 2)
    total_features = sampled_data.shape[2]
    downsampled_data = np.empty(shape=(0, samples_per_batch, total_features))
    
    #In Skip technique, we skip 2 frames consecutively.
    if downsample_technique == 'skip':
        downsampled_data = sampled_data[:,::2,:]
    else:
        #Iterating over batches
        for batch in sampled_data:
    
            averaged_batch = np.empty(shape=(0, total_features))
    
            #In each iteration, averaging 2 Frames and appending it to the variable
            for i in range(0, batch.shape[0], 2):
                averaged_batch = np.append(averaged_batch, np.mean(batch[i:i+2, :], axis = 0).reshape((1, total_features)), axis = 0)
            
            #Appending the whole batched averaged downsampled data to the new variable created before
            downsampled_data = np.append(downsampled_data, averaged_batch.reshape((1, samples_per_batch, total_features)), axis = 0)
    
    return downsampled_data

### Adding more preprocessing steps (Normalization and gussian noise)

In [6]:
def preprocess_data(sampled_dataX, sampled_dataY, normalize=True, add_noise=True
                    , stddev=0.05) :
    """
    Function to preprocess data by normalizing input features and adding guassian
    noise to increase model robustness
    """  
    if normalize :
        sampled_dataX =  tf.keras.utils.normalize(sampled_dataX, axis=2)
    
    if add_noise :
        guassian_noise_layer = tf.keras.layers.GaussianNoise(stddev=stddev)
        sampled_dataX = guassian_noise_layer(sampled_dataX)
    
    return sampled_dataX, sampled_dataY

## Defining different components of the model

### Defining the Interpolation Layer

In [7]:
class InterpolationLayer(tf.keras.layers.Layer):
    """
    Custom interpolation layer extending the keras layer class
    it has one attribute num_frames to be interpolated between each two consecutive 
    timesteps
    it has one main function interpolateFrames  
    """
  
    def __init__(self, num_frames=5):
        super(InterpolationLayer, self).__init__()
        self.num_frames = num_frames

    def interpolateFrames(self, inputs):
        """
        Takes input tensors of shape(batch_size, timesteps, features)
        returns interpolated frames with shape(batch_size, timesteps*num_frames, features)
        """
        
        batch_size = inputs.shape[0]
        timesteps = inputs.shape[1]
        features = inputs.shape[2]
        interpolated_frames = tf.zeros([0, features])
        
        for batch in tf.range(tf.shape(inputs)[0]) :
            tf.autograph.experimental.set_loop_options(
            shape_invariants=[(interpolated_frames, tf.TensorShape([None, features]))])
            for t in range(timesteps) :
                for j in range(self.num_frames) :
                    X_i0 = inputs[batch, t]
                    if(t == timesteps-1) :
                        X_i1 = inputs[batch, t]
                    else :  
                        X_i1 = inputs[batch, t+1]
                    alpha_j = j/self.num_frames
                    current_frame = alpha_j*X_i0 + (1-alpha_j)*X_i1
                    current_frame = tf.reshape(current_frame, [1, features])
                    interpolated_frames = tf.concat((interpolated_frames, current_frame), axis=0)

        interpolated_frames = tf.reshape(interpolated_frames,[tf.shape(inputs)[0], (timesteps)*self.num_frames, features])
        return interpolated_frames

    def call(self, inputs):
        return self.interpolateFrames(inputs)

### Trying to create the Keras GlocalNet model through a custom function

In [8]:
def create_glocalNet_model(enocder_hidden_state=200, decoder_hidden_state=200, 
                 output_diminsion=64, input_diminsions=74, LSTM_dropout=0.25, dense_activation='relu',
                 interpolation_frames=5, exclude_locgen=False) :
    #Glogen encoder
    encoder_inputs = Input(shape=(10, input_diminsions))
    encoder = Bidirectional(LSTM(enocder_hidden_state, return_sequences=True, return_state=True))
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    #Glogen decoder
    decoder_lstm = Bidirectional(LSTM(decoder_hidden_state, return_sequences=True, return_state=True))
    decoder_outputs, _, _ = decoder_lstm(encoder_outputs,
                                        initial_state=encoder_states)
    decoder_dense = TimeDistributed(Dense(output_diminsion, activation=dense_activation))
    decoder_outputs = decoder_dense(decoder_outputs)

    #Interpolation layer
    interpolation_layer = InterpolationLayer(num_frames=interpolation_frames)
    interpolation_output = interpolation_layer(decoder_outputs)
    #return the model if execlude_locgen is true
    if(exclude_locgen) :
        return Model_(encoder_inputs, interpolation_output)
    
    #Locgen encoder
    encoder_locgen = Bidirectional(LSTM(enocder_hidden_state, return_sequences=True, return_state=True))
    encoder_outputs_locgen, state_h_locgen, state_c_locgen = encoder_locgen(interpolation_output)
    encoder_states_locgen = [state_h_locgen, state_c_locgen]
    #Locgen decoder
    decoder_lstm_locgen = Bidirectional(LSTM(decoder_hidden_state, return_sequences=True, return_state=True))
    decoder_outputs_locgen, _, _ = decoder_lstm(encoder_outputs_locgen,
                                        initial_state=encoder_states_locgen)
    decoder_dense_locgen = TimeDistributed(Dense(output_diminsion, activation=dense_activation))
    glocalNet_output = decoder_dense_locgen(decoder_outputs_locgen)
    return Model_(encoder_inputs, glocalNet_output)


### Custom GlocalNet class

In [9]:
class GlocalNet(Model_):
    """
    A full GlocalNet implementation include the three main stages
    Glogen generating initial sparse frames
    Interpolation layer generating dense frames from Glogen output
    Locgen generating the final output by smoothing the interpolated frames
    """
    def __init__(self, enocder_hidden_state=200, decoder_hidden_state=200, 
                 output_diminsion=64, LSTM_dropout=0.25, dense_activation='relu',
                 interpolation_frames=5, exclude_locgen=False, only_glogen = False,
                 self_attention = False, cross_attention = False):
        super(GlocalNet, self).__init__()
        
        assert ~(self_attention and cross_attention), f'only one type of attention can be used' 
        
        self.exclude_locgen = exclude_locgen
        self.only_glogen = only_glogen
        self.self_attention = self_attention
        self.cross_attention = cross_attention
        
        #Glogen layers
        self.glogen_encoder = Bidirectional(LSTM(enocder_hidden_state, return_state=True
                                   , return_sequences=True, dropout=LSTM_dropout))
        self.glogen_decoder = Bidirectional(LSTM(decoder_hidden_state, return_sequences=True,
                                   return_state=True, dropout=LSTM_dropout))
        #Locgen layers
        self.locgen_encoder = Bidirectional(LSTM(enocder_hidden_state, return_sequences=True,
                                   return_state=True, dropout=LSTM_dropout))
        self.locgen_decoder = Bidirectional(LSTM(decoder_hidden_state, return_sequences=True,
                                   return_state=True, dropout=LSTM_dropout))
        #Glogen dense layer
        self.glogen_dense_layer = TimeDistributed(Dense(output_diminsion,
                                                        activation=dense_activation)) 
        #Interpolation layer
        self.interpolation_layer = InterpolationLayer(num_frames=interpolation_frames)
        #Locgen dense layer
        self.locgen_dense_layer = TimeDistributed(Dense(output_diminsion,
                                                        activation=dense_activation)) 
        
        #Attention layer to be used in both self and cross attention cases
        self.attention_layer = tf.keras.layers.Attention()
        
    def call(self, inputs):
        #Glogen calls      
        encoder_outputs, fwd_state_h, fwd_state_c ,back_state_h, back_state_c= self.glogen_encoder(inputs)
        encoder_states = [fwd_state_h, fwd_state_c ,back_state_h, back_state_c]
        
        #Cross attention case
        if self.cross_attention :
            decoder_output,_,_,_,_ = self.glogen_decoder(encoder_outputs, initial_state=encoder_states)
            attention_score = self.attention_layer(inputs = [decoder_output, encoder_outputs])
            decoder_output,_,_,_,_ = self.glogen_decoder(attention_score, initial_state=encoder_states)
        
        #Self attention case
        elif self.self_attention:
            glogen_decoder_input = self.attention_layer(inputs = [encoder_outputs, encoder_outputs])
            decoder_output,_,_,_,_ = self.glogen_decoder(glogen_decoder_input, initial_state=encoder_states)
            
        #Basic case without attention
        else :
            decoder_output,_,_,_,_ = self.glogen_decoder(encoder_outputs, initial_state=encoder_states)
        
        glogen_output = self.glogen_dense_layer(decoder_output)
        
        if self.only_glogen:
            return glogen_output

        #Interpolation call
        interpolated_frames = self.interpolation_layer(glogen_output)
        
        if self.exclude_locgen :
            return interpolated_frames

        #Locgen calls
        locgen_encoder_outputs, locgen_fwd_state_h, locgen_fwd_state_c,locgen_back_state_h, locgen_back_state_c = self.locgen_encoder(interpolated_frames)
        locgen_encoder_states = [locgen_fwd_state_h, locgen_fwd_state_c,locgen_back_state_h, locgen_back_state_c]
        locgen_output, _, _,_,_  = self.locgen_decoder(locgen_encoder_outputs, initial_state=locgen_encoder_states)
        final_output = self.locgen_dense_layer(locgen_output)
        return final_output

### Action Classifier Model

In [10]:
class ActionClassifier(Model_):
    def __init__(self):
        super(ActionClassifier, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv1D(filters=128, kernel_size=8)
        self.conv2 = tf.keras.layers.Conv1D(filters=256, kernel_size=5)
        self.conv3 = tf.keras.layers.Conv1D(filters=128, kernel_size=3)
        self.denseLayer = tf.keras.layers.Dense(10, activation="softmax")
        self.batchNormalization1 = tf.keras.layers.BatchNormalization()
        self.batchNormalization2 = tf.keras.layers.BatchNormalization()
        self.batchNormalization3 = tf.keras.layers.BatchNormalization()
        self.globalAveragePooling = tf.keras.layers.GlobalAveragePooling1D()
        self.relu1 = tf.keras.layers.ReLU()
        self.relu2 = tf.keras.layers.ReLU()
        self.relu3 = tf.keras.layers.ReLU()
        
    def call(self, input):      
        output = self.conv1(input)
        output = self.batchNormalization1(output)
        output = self.relu1(output)
        
        output = self.conv2(output)
        output = self.batchNormalization2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.batchNormalization3(output)
        output = self.relu3(output)
        
        output = self.globalAveragePooling(output)
        output = self.denseLayer(output)
        
        return output

### Defining different Types of Loss functions

In [11]:
class Loss() :
    """
    Joint loss class with two weight attributes for two different losses
    first one is the loss joint and the second is the loss_motion_flow
    """
    
    def __init__(self, lambda1=0.5, lambda2=0.5) :
        self.lambda1 = lambda1
        self.lambda2 = lambda2

    def loss_joint(self, predicted_sequance_batch, target_sequance_batch) :
        """
        Loss between the joint positions and its corresponding counterparts in the groundtruth
        """
        diff_norm_2 = tf.math.reduce_sum(tf.square(tf.subtract(predicted_sequance_batch, target_sequance_batch)), axis=2)
        return tf.reduce_sum(diff_norm_2, axis=1) 

    def loss_motion_flow(self, predicted_sequance_batch, target_sequance_batch) :
        """
        Loss between the motion flow of predicted sequance and the ground truth
        where the motion flow is the euclidean distance between each two consecutive frames
        """
        predictions_tomporal_diffs = tf.experimental.numpy.diff(predicted_sequance_batch, axis=1)
        real_tomporal_diffs = tf.experimental.numpy.diff(target_sequance_batch, axis=1)
        prediction_motion_flow_diff_norm_2 = tf.reduce_sum(tf.square(tf.subtract(predictions_tomporal_diffs, real_tomporal_diffs)), axis=2)
        return tf.reduce_sum(prediction_motion_flow_diff_norm_2, axis=1)

    def total_loss(self, target_sequance_batch, predicted_sequance_batch) :
        """
        calculating the total loss through a combination of the joint_loss and motion_flow_loss
        """
        joints_loss = self.loss_joint(predicted_sequance_batch, target_sequance_batch)
        motion_flow_loss = self.loss_motion_flow(predicted_sequance_batch, target_sequance_batch)
        return self.lambda1*joints_loss + self.lambda2*motion_flow_loss

### Defining different types of Evaluation Metrics

In [12]:
class Metrics():
    """
    A class containing different types of Evaluation Metrics.
    """
    
    def __init__(self, mmd_kernel='gaussian') :
        self.mmd_kernel = mmd_kernel
    
    def custom_sequence_MMD_loss(self, target_sequance_batch, predicted_sequance_batch):
        """
        Calculating the Sequence MMD Loss between prediction and the ground Truth.
        Additionally combining the last two dimensions 
        """
        mmd_loss = MMD.MMDLoss(kernel=self.mmd_kernel)
        
        total_batches = predicted_sequance_batch.shape[0]
        frames_per_batch = predicted_sequance_batch.shape[1] * predicted_sequance_batch.shape[2]
        
        return mmd_loss(tf.reshape(predicted_sequance_batch, [total_batches, frames_per_batch]),
                        tf.reshape(target_sequance_batch, [total_batches, frames_per_batch]))
        
    def MPJPE2(self, y_true, y_pred, number_of_joints = 32):
        """
        Calculating the Mean Per Joint Position Error (MPJPE) between prediction and the ground Truth.
        """
        yt= y_true.reshape((-1,number_of_joints,2))
        yp= y_pred.reshape((-1,number_of_joints,2))
        dist= np.zeros(yt.shape[0])
        
        for i in range(yt.shape[0]):
            dist[i] = np.linalg.norm(yt[i] - yp[i])
        
        return np.mean(dist)
    
    def NPSS(self, euler_gt_sequences, euler_pred_sequences):
        """
        A function to compute the Normalized Power Spectrum Similarity (NPSS) metric between predictions and the ground Truth [2] and [3].
        """        
        # computing 1) fourier coeffs 2)power of fft 3) normalizing power of fft dim-wise 4) cumsum over freq. 5) EMD 
        gt_fourier_coeffs = np.zeros(euler_gt_sequences.shape, dtype = 'complex_')
        pred_fourier_coeffs = np.zeros(euler_pred_sequences.shape, dtype = 'complex_')

        # power vars
        gt_power = np.zeros((gt_fourier_coeffs.shape))
        pred_power = np.zeros((gt_fourier_coeffs.shape))

        # normalizing power vars
        gt_norm_power = np.zeros(gt_fourier_coeffs.shape)
        pred_norm_power = np.zeros(gt_fourier_coeffs.shape)

        cdf_gt_power = np.zeros(gt_norm_power.shape)
        cdf_pred_power = np.zeros(pred_norm_power.shape)

        emd = np.zeros(cdf_pred_power.shape[0:3:2])

        # used to store powers of feature_dims and sequences used for avg later
        seq_feature_power = np.zeros(euler_gt_sequences.shape[0:3:2])
        power_weighted_emd = 0

        for s in range(euler_gt_sequences.shape[0]):

            for d in range(euler_gt_sequences.shape[2]):
                gt_fourier_coeffs[s,:,d] = np.fft.fft(euler_gt_sequences[s,:,d]) # slice is 1D array
                pred_fourier_coeffs[s,:,d] = np.fft.fft(euler_pred_sequences[s,:,d])

                # computing power of fft per sequence per dim
                gt_power[s,:,d] = np.square(np.absolute(gt_fourier_coeffs[s,:,d]))
                pred_power[s,:,d] = np.square(np.absolute(pred_fourier_coeffs[s,:,d]))

                # matching power of gt and pred sequences
                gt_total_power = np.sum(gt_power[s,:,d])
                pred_total_power = np.sum(pred_power[s,:,d])
                #power_diff = gt_total_power - pred_total_power

                # adding power diff to zero freq of pred seq
                #pred_power[s,0,d] = pred_power[s,0,d] + power_diff

                # computing seq_power and feature_dims power 
                seq_feature_power[s,d] = gt_total_power

                # normalizing power per sequence per dim
                if gt_total_power != 0:
                    gt_norm_power[s,:,d] = gt_power[s,:,d] / gt_total_power 

                if pred_total_power !=0:
                    pred_norm_power[s,:,d] = pred_power[s,:,d] / pred_total_power

                # computing cumsum over freq
                cdf_gt_power[s,:,d] = np.cumsum(gt_norm_power[s,:,d]) # slice is 1D
                cdf_pred_power[s,:,d] = np.cumsum(pred_norm_power[s,:,d])

                # computing EMD 
                emd[s,d] = np.linalg.norm((cdf_pred_power[s,:,d] - cdf_gt_power[s,:,d]), ord=1)

        # computing weighted emd (by sequence and feature powers)
        power_weighted_emd = np.average(emd, weights=seq_feature_power) 

        return power_weighted_emd

### A function to start the experiment of training and evaluating the model

In [13]:
def run_experiment(sampled_dataX, sampled_dataY, learning_rate=0.002, lambda1=0.5,
                   lambda2=0.5, use_mse=False, use_MMD=False, metrics=None, output_diminsion=64,
                   batch_size=100, epochs=50, validation_split=0.2, activation="relu",
                   dropout=0.25, exclude_locgen=False, interpolate_frames = 5, only_glogen = False,
                   self_attention = False, cross_attention = False) :
    """
    Method takes all hyperparameters as input paramters and returns the model and history as
    a result
    """
    glocal_model = GlocalNet(dense_activation=activation, LSTM_dropout=dropout,
                             exclude_locgen=exclude_locgen, only_glogen=only_glogen,
                             output_diminsion=output_diminsion, 
                             interpolation_frames = interpolate_frames,
                             self_attention = self_attention, cross_attention = cross_attention)
    if use_mse :
        loss_function = tf.keras.losses.mean_squared_error
    elif use_MMD :
        loss_function = Loss().custom_sequence_MMD_loss
    else :
        loss_function = Loss(lambda1=lambda1, lambda2=lambda2).total_loss

    glocal_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                         loss=loss_function, metrics=metrics, run_eagerly=False)
    
    history = glocal_model.fit(sampled_dataX, sampled_dataY, batch_size=batch_size, 
                               epochs=epochs, validation_split=validation_split)
    
    return history, glocal_model

### A function to Resume the experiment of training and evaluating the model

In [14]:
def resume_training(glocal_model, sampled_dataX, sampled_dataY, learning_rate=0.002, lambda1=0.5,
                   lambda2=0.5, use_mse=False, use_MMD=False, metrics=None,
                   batch_size=100, epochs=50, validation_split=0.2) :
    """
    function to resume training of a model
    """
    if use_mse :
        loss_function = tf.keras.losses.mean_squared_error
    elif use_MMD :
        loss_function = Loss().custom_sequence_MMD_loss
    else :
        loss_function = Loss(lambda1=lambda1, lambda2=lambda2).total_loss

    glocal_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss=loss_function, metrics=metrics)
    
    history = glocal_model.fit(sampled_dataX, sampled_dataY,
                            batch_size=batch_size,
                            epochs=epochs, validation_split=validation_split)
    
    return history, glocal_model

### A function to Train Action Classifier

In [15]:
def train_action_classifier(data, label, 
                            optimizer = tf.keras.optimizers.Adam(0.001), 
                            loss = tf.keras.losses.CategoricalCrossentropy(), 
                            metrics = [tf.keras.metrics.CategoricalAccuracy()], 
                            epochs = 500, batch_size = 32, validation_split = 0.2):
    """
    The function trains the Action Classifier and evaluate it on the dataset.
    """
    print('Starting to Train Action Classifier...')
    
    #Initializing the action classifier model
    action_classifer = ActionClassifier()
    
    #Compiling the model
    action_classifer.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    #Fitting the model on the dataset
    action_classifer.fit(data, label, epochs=epochs, batch_size=batch_size, verbose = 1, 
                         validation_split=validation_split, 
                         callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001)])
    
    print('\nTraining Complete!\n')
    
    #Evaluating the action classifier
    score = action_classifer.evaluate(data, label)
    print('Action Classifier Evaluation:\nLoss: {}\nAccuracy: {}'.format(score[0], score[1]))
    
    #Returning the trained action classifier
    return action_classifer

### Functions to visualize certain frames from the Dataset

In [16]:
def create_gif(lenght = 10, path_to_save = ''):
    """
    Function to create and save a GIF from different number of frames
    """
    list=[]
    for l in range(lenght):
        list.append(f'{path_to_save}_frame{l}.png')

    with imageio.get_writer(f'{path_to_save}.gif', mode='I', duration=0.1) as writer:
        for filename in list:
            image = imageio.imread(filename)
            writer.append_data(image)

In [17]:
def visualize_frames(sample, dynamic_joints_only=False, num_frames_to_visualize=10, 
                    path_to_save="", save_gif=False,
                    joints_to_ignore=[4,5,9,10,11,16,20,21,22,23,24,28,29,30,31]) :
    """
    Visualization function to draw a certain number of frames in a given sample
    ignoring the joints mentioned in joints_to_ignore array
    """
    assert num_frames_to_visualize <= sample.shape[0], f"number of frames should be less than or equal to the total frames in the sample"
    
    fig, axs = plt.subplots(ncols=num_frames_to_visualize, figsize=(40, 10))
    fig.tight_layout(pad=1.0)
    
    for t in range(num_frames_to_visualize) :
        #Removing unnecessary joints for visualization
        if(dynamic_joints_only) :
            #Check if no joints needs to be removed
            truncated_frame = sample[t]
        else :
            #Removing the joints based on joints_to_ignore
            joints_to_ignore_2d = [element * 2 for element in joints_to_ignore]
            for i in range(len(joints_to_ignore_2d)) :
                joints_to_ignore_2d.append(joints_to_ignore_2d[i]+1)
            truncated_frame = np.delete(sample[t], joints_to_ignore_2d)   

        #In case of including only moving joints for Human3.6M(17 joints)      
        x_axis_array = truncated_frame[0:34:2]
        y_axis_array = truncated_frame[1:35:2]
        #Scattering all the 17 joints
        axs[t].scatter(x_axis_array, y_axis_array)
        #Plotting right leg
        axs[t].plot(x_axis_array[:4], y_axis_array[:4], "tab:blue")
        #plotting left leg
        axs[t].plot(x_axis_array[[0, 4, 5, 6]], y_axis_array[[0, 4, 5, 6]])
        #plotting from hip to head
        axs[t].plot(x_axis_array[[0, 7, 8, 9, 10]], y_axis_array[[0, 7, 8, 9, 10]])
        #plotting from neck to left shoulder
        axs[t].plot(x_axis_array[[9, 11, 12, 13]], y_axis_array[[9, 11, 12, 13]])
        #plotting from neck to right shoulder
        axs[t].plot(x_axis_array[[9, 14, 15, 16]], y_axis_array[[9, 14, 15, 16]])
        axs[t].invert_yaxis()
        axs[t].set_xticks([])
        axs[t].set_yticks([])
        
        if(len(path_to_save) > 0) :
            extent = axs[t].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
            fig.savefig(f'{path_to_save}_frame{t}.png', bbox_inches=extent)
    
    if save_gif==True:
        create_gif(lenght=num_frames_to_visualize,path_to_save=path_to_save)

    plt.show()

### Simple Baseline Model for short term predictions (10 frames)

In [18]:
class BasicRNN(Model_):
    """
    simple many to many RNN model
    """
    def __init__(self, output_diminsions=64, RNN_type="classical", activation="relu"):
        super(BasicRNN, self).__init__()
        possible_RNN_types = ["classical", "LSTM", "GRU"]
        assert RNN_type in possible_RNN_types, f"RNN_type should be one of the valid values ['classical', 'LSTM', 'GRU']"
        if(RNN_type == "classical") :
            self.ruccernt_layer = SimpleRNN(output_diminsions, return_sequences=True)
        elif(RNN_type == "LSTM") : 
            self.ruccernt_layer = LSTM(output_diminsions, return_sequences=True)
        else :
            self.ruccernt_layer = GRU(output_diminsions, return_sequences=True)
        self.dense_layer = TimeDistributed(Dense(output_diminsions,
                                                        activation=activation)) 

    def call(self, input) :
        output = self.ruccernt_layer(input)
        output = self.dense_layer(output)
        return output        

### Functions to save Custom Models

In [19]:
def saveGlocalNetWeights(model, file_path) :
    """
    A function to save all layers weights except for interpolation layer
    """
    model_layers = np.array([], dtype=object)
    for layer in model.layers :
        model_layers = np.append(model_layers, layer.get_weights())
    np.save(file_path, model_layers)

In [20]:
def createGlocalNetFromFile(model : GlocalNet, model_weights_file, sample_input, output_diminsions=64,
interpolation_frames=2, dense_activation="relu") :
    """
    function use presaved weights from file_path(model_weights_file) and uses sample_input to 
    build the model
    """
    #Create an empty glocalNet model
    model = GlocalNet(exclude_locgen=model.exclude_locgen, output_diminsion=output_diminsions,
     interpolation_frames=interpolation_frames, dense_activation=dense_activation)
    model_weights = np.load(model_weights_file, allow_pickle=True)
    model(sample_input)
    if(model.exclude_locgen) :
        #Setting Glogen layers
        layer0_weights = model_weights[:3]
        layer1_weights = model_weights[3:6]
        layer4_weights = model_weights[6:]
        #Setting the Glogen layers
        model.layers[0].set_weights(layer0_weights)
        model.layers[1].set_weights(layer1_weights)
        model.layers[4].set_weights(layer4_weights)
    else :    
        #Setting Glogen layers
        layer0_weights = model_weights[:3]
        layer1_weights = model_weights[3:6]
        layer4_weights = model_weights[12:14]
        #Setting Locgen layers
        layer2_weights = model_weights[6:9]
        layer3_weights = model_weights[9:12]
        layer6_weights = model_weights[14:]
            #Building the model to be able to set the layers' weights
        model(sample_input)
        #Setting the Glogen layers
        model.layers[0].set_weights(layer0_weights)
        model.layers[1].set_weights(layer1_weights)
        model.layers[4].set_weights(layer4_weights)
        #Setting locgen layers
        model.layers[2].set_weights(layer2_weights)
        model.layers[3].set_weights(layer3_weights)
        model.layers[6].set_weights(layer6_weights)
        
    return model

## Experimentations

### Reading and Preprocessing the Dataset

In [21]:
#Reading the whole dataset for shortterm predictions
sampled_data_short_term = Dataset_loading('./H3.6csv', sample_size=20, include_movable_joints=False).read_dataset()

In [22]:
#Reading only the movable dataset points for shortterm predictions
sampled_data_short_term_movable = Dataset_loading('./H3.6csv', sample_size=20, include_movable_joints=True).read_dataset()

In [23]:
#For long term prediction, we need a sample size of 60(10 frames input sequance, 50 frames predicted sequance)
sampled_data_all = Dataset_loading('./H3.6csv', sample_size=60, include_movable_joints=False).read_dataset()

In [24]:
#For long term movable joints prediction, we need a sample size of 60(10 frames input sequance, 50 frames predicted sequance)
sampled_data_movable = Dataset_loading('./H3.6csv', sample_size=60, include_movable_joints=True).read_dataset()

### Reading the Dataset for Action Classifier

In [25]:
sampled_data_classifier, sampled_labels_classifier = Dataset_loading('./H3.6csv', sample_size=60, include_movable_joints=False, include_action_labels=False, return_action_labels=True).read_dataset()

### Splitting the Dataset into Features and Labels

In [26]:
sampled_dataX_all, sampled_dataY_all = split_to_features_labels(sampled_data_all, input_sequance_size=10, total_features=64)

In [27]:
sampled_dataX_movable, sampled_dataY_movable = split_to_features_labels(sampled_data_movable, input_sequance_size=10, total_features=34)

In [28]:
sampled_dataX_short_term, sampled_dataY_short_term = split_to_features_labels(sampled_data_short_term, input_sequance_size=10, total_features=64)

In [29]:
sampled_dataX_short_term_movable, sampled_dataY_short_term_movable = split_to_features_labels(sampled_data_short_term_movable, input_sequance_size=10, total_features=34)

In [30]:
print('Dataset with all features(short term)')
print('Total Samples: {}\nTotal Frames: {}\nTotal Features: {}'.format(sampled_dataY_short_term.shape[0],
                                                                       sampled_dataY_short_term.shape[1],
                                                                       sampled_dataY_short_term.shape[2]))

Dataset with all features(short term)
Total Samples: 77140
Total Frames: 10
Total Features: 64


In [31]:
print('Dataset with Movable features(short term)')
print('Total Samples: {}\nTotal Frames: {}\nTotal Features: {}'.format(sampled_dataY_short_term_movable.shape[0],
                                                                       sampled_dataY_short_term_movable.shape[1],
                                                                       sampled_dataY_short_term_movable.shape[2]))

Dataset with Movable features(short term)
Total Samples: 77140
Total Frames: 10
Total Features: 34


In [32]:
print('Dataset with all features')
print('Total Samples: {}\nTotal Frames: {}\nTotal Features: {}'.format(sampled_dataY_all.shape[0],
                                                                       sampled_dataY_all.shape[1],
                                                                       sampled_dataY_all.shape[2]))

Dataset with all features
Total Samples: 25520
Total Frames: 50
Total Features: 64


In [33]:
print('Dataset with only movable features')
print('Total Samples: {}\nTotal Frames: {}\nTotal Features: {}'.format(sampled_dataY_movable.shape[0],
                                                                       sampled_dataY_movable.shape[1],
                                                                       sampled_dataY_movable.shape[2]))

Dataset with only movable features
Total Samples: 25520
Total Frames: 50
Total Features: 34


### Adding Noise and Downsampling to improve model performance and robustness

#### Preprocessed and Downsampled Datasets

In [34]:
downsampled_data_all = downsampling(sampled_data_all, 'skip')
downsampled_data_movable = downsampling(sampled_data_movable, 'skip')
downsampled_data_shortterm_movable = downsampling(sampled_data_short_term_movable, 'skip')

In [35]:
downsampled_dataX_all, downsampled_dataY_all = split_to_features_labels(downsampled_data_all, input_sequance_size=10, total_features=64)
preprocessed_downsampled_dataX_all, preprocessed_downsampled_dataY_all = preprocess_data(downsampled_dataX_all, downsampled_dataY_all)

In [36]:
downsampled_dataX_movable, downsampled_dataY_movable = split_to_features_labels(downsampled_data_movable, input_sequance_size=10, total_features=34)
preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable = preprocess_data(downsampled_dataX_movable, downsampled_dataY_movable)

In [37]:
downsampled_dataX_shortterm_movable, downsampled_dataX_shortterm_movable = split_to_features_labels(downsampled_data_shortterm_movable, input_sequance_size=5, total_features=34)
preprocessed_downsampled_dataX_shortterm_movable, preprocessed_downsampled_dataY_shortterm_movable = preprocess_data(downsampled_dataX_shortterm_movable, downsampled_dataX_shortterm_movable)

### Training Action Classifier

In [38]:
action_classifer = train_action_classifier(sampled_data_classifier, sampled_labels_classifier)

Starting to Train Action Classifier...
Epoch 1/500
638/638 [==============================] - 22s 12ms/step - loss: 1.4815 - categorical_accuracy: 0.4536 - val_loss: 3.0594 - val_categorical_accuracy: 0.2220 - lr: 0.0010
Epoch 2/500
638/638 [==============================] - 7s 12ms/step - loss: 1.2516 - categorical_accuracy: 0.5394 - val_loss: 1.8150 - val_categorical_accuracy: 0.3474 - lr: 0.0010
Epoch 3/500
638/638 [==============================] - 7s 11ms/step - loss: 1.1409 - categorical_accuracy: 0.5773 - val_loss: 2.2221 - val_categorical_accuracy: 0.3335 - lr: 0.0010
Epoch 4/500
638/638 [==============================] - 7s 11ms/step - loss: 1.0644 - categorical_accuracy: 0.6150 - val_loss: 2.7034 - val_categorical_accuracy: 0.2735 - lr: 0.0010
Epoch 5/500
638/638 [==============================] - 7s 12ms/step - loss: 1.0124 - categorical_accuracy: 0.6321 - val_loss: 2.6622 - val_categorical_accuracy: 0.2855 - lr: 0.0010
Epoch 6/500
638/638 [==============================] - 

In [39]:
action_classifer.save('./actionClassifier', save_format='tf')

INFO:tensorflow:Assets written to: ./actionClassifier\assets


INFO:tensorflow:Assets written to: ./actionClassifier\assets


### Running experiments with different hyperparameters

#### Experimentations using Simple Multivariate Time Series Models (Simple RNN, LSTM, GRU)

##### Running simple LSTM model with Relu

In [ ]:
simple_RNN1 = BasicRNN(RNN_type="LSTM")

In [ ]:
history_simple_RNN1, simple_RNN1 = resume_training(simple_RNN1, sampled_dataX_short_term, sampled_dataY_short_term, epochs=50, batch_size=20, use_mse=True, validation_split=0.0,
                                                  metrics=[tf.keras.losses.mean_absolute_percentage_error, Metrics().custom_sequence_MMD_loss])

In [ ]:
simple_RNN1_pred = simple_RNN1.predict(sampled_dataX_short_term)
rnn1_npss = Metrics().NPSS(sampled_dataY_short_term, simple_RNN1_pred)
rnn1_mpjpe = Metrics().MPJPE2(sampled_dataY_short_term, simple_RNN1_pred)

print('Simple LSTM Model with RELU activation:\nNPSS: {}\nMPJPE: {}'.format(rnn1_npss, rnn1_mpjpe))

##### Running simple GRU model with Relu

In [ ]:
simple_RNN2 = BasicRNN(RNN_type="GRU")

In [ ]:
history_simple_RNN2, simple_RNN2 = resume_training(simple_RNN2, sampled_dataX_short_term, sampled_dataY_short_term, epochs=10, batch_size=20, use_mse=True, validation_split=0.0,
                                                  metrics=[tf.keras.losses.mean_absolute_percentage_error, Metrics().custom_sequence_MMD_loss])

In [ ]:
simple_RNN2_pred = simple_RNN2.predict(sampled_dataX_short_term)
rnn2_npss = Metrics().NPSS(sampled_dataY_short_term, simple_RNN2_pred)
rnn2_mpjpe = Metrics().MPJPE2(sampled_dataY_short_term, simple_RNN2_pred)

print('Simple GRU Model with RELU activation:\nNPSS: {}\nMPJPE: {}'.format(rnn2_npss, rnn2_mpjpe))

##### Running simple RNN model with Relu

In [ ]:
simple_RNN3 = BasicRNN()

In [ ]:
history_simple_RNN3, simple_RNN3 = resume_training(simple_RNN3, sampled_dataX_short_term, sampled_dataY_short_term, epochs=50, batch_size=20, use_mse=True, validation_split=0.0,
                                                  metrics=[tf.keras.losses.mean_absolute_percentage_error, Metrics().custom_sequence_MMD_loss])

In [ ]:
simple_RNN3_pred = simple_RNN3.predict(sampled_dataX_short_term)
rnn3_npss = Metrics().NPSS(sampled_dataY_short_term, simple_RNN3_pred)
rnn3_mpjpe = Metrics().MPJPE2(sampled_dataY_short_term, simple_RNN3_pred)

print('Simple RNN Model with RELU activation:\nNPSS: {}\nMPJPE: {}'.format(rnn3_npss, rnn3_mpjpe))

##### Running simple RNN model with linear activation

In [ ]:
simple_RNN4 = BasicRNN(activation="linear")

In [ ]:
history_simple_RNN4, simple_RNN4 = resume_training(simple_RNN4, sampled_dataX_short_term, sampled_dataY_short_term, epochs=50, batch_size=20, use_mse=True, validation_split=0.0,
                                                  metrics=[tf.keras.losses.mean_absolute_percentage_error, Metrics().custom_sequence_MMD_loss])

In [ ]:
simple_RNN4_pred = simple_RNN4.predict(sampled_dataX_short_term)
rnn4_npss = Metrics().NPSS(sampled_dataY_short_term, simple_RNN4_pred)
rnn4_mpjpe = Metrics().MPJPE2(sampled_dataY_short_term, simple_RNN4_pred)

print('Simple RNN Model with Linear activation:\nNPSS: {}\nMPJPE: {}'.format(rnn4_npss, rnn4_mpjpe))

##### Running simple LSTM model with linear activation

In [ ]:
simple_RNN5 = BasicRNN(RNN_type="LSTM", activation="linear")

In [ ]:
history_simple_RNN5, simple_RNN5 = resume_training(simple_RNN5, sampled_dataX_short_term, sampled_dataY_short_term, epochs=50, batch_size=20, use_mse=True, validation_split=0.0,
                                                  metrics=[tf.keras.losses.mean_absolute_percentage_error, Metrics().custom_sequence_MMD_loss])

In [ ]:
simple_RNN5_pred = simple_RNN5.predict(sampled_dataX_short_term)
rnn5_npss = Metrics().NPSS(sampled_dataY_short_term, simple_RNN5_pred)
rnn5_mpjpe = Metrics().MPJPE2(sampled_dataY_short_term, simple_RNN5_pred)

print('Simple LSTM Model with Linear activation:\nNPSS: {}\nMPJPE: {}'.format(rnn5_npss, rnn5_mpjpe))

##### Running simple GRU model with linear activation

In [ ]:
simple_RNN6 = BasicRNN(RNN_type="GRU", activation="linear")

In [ ]:
history_simple_RNN6, simple_RNN6 = resume_training(simple_RNN6, sampled_dataX_short_term, sampled_dataY_short_term, epochs=50, batch_size=20, use_mse=True, validation_split=0.0,
                                                  metrics=[tf.keras.losses.mean_absolute_percentage_error, Metrics().custom_sequence_MMD_loss])

In [ ]:
simple_RNN6_pred = simple_RNN6.predict(sampled_dataX_short_term)
rnn6_npss = Metrics().NPSS(sampled_dataY_short_term, simple_RNN6_pred)
rnn6_mpjpe = Metrics().MPJPE2(sampled_dataY_short_term, simple_RNN6_pred)

print('Simple GRU Model with Linear activation:\nNPSS: {}\nMPJPE: {}'.format(rnn6_npss, rnn6_mpjpe))

#### GlocalNet Base Implementation Experiments

##### Running Experiment on Base Paper with Downsampled movable datapoints and MSE loss

In [ ]:
history_model1, glocal_model1 = run_experiment(preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable, batch_size=20, dropout=0.0, epochs=1, 
                                               use_mse=True, output_diminsion=34, validation_split=0.2, interpolate_frames=2, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model1_pred = glocal_model1.predict(preprocessed_downsampled_dataX_movable)
glocal_model1_npss = Metrics().NPSS(preprocessed_downsampled_dataY_movable, glocal_model1_pred)
glocal_model1_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_movable, glocal_model1_pred, number_of_joints=17)

print('Model 1:\nNPSS: {}\nMPJPE: {}'.format(glocal_model1_npss, glocal_model1_mpjpe))

##### Running Experiment on Base Paper with Downsampled movable datapoints and Joint loss

In [ ]:
history_model2, glocal_model2 = run_experiment(preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable, batch_size=20, dropout=0.0, epochs=50, 
                                               output_diminsion=34, validation_split=0.2, interpolate_frames=2, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model2_pred = glocal_model2.predict(preprocessed_downsampled_dataX_movable)
glocal_model2_npss = Metrics().NPSS(preprocessed_downsampled_dataY_movable, glocal_model2_pred)
glocal_model2_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_movable, glocal_model2_pred, number_of_joints=17)

print('Model 2:\nNPSS: {}\nMPJPE: {}'.format(glocal_model2_npss, glocal_model2_mpjpe))

##### Running Experiment with ONLY Glogen model and MSE Loss

In [ ]:
history_model3, glocal_model3 = run_experiment(preprocessed_downsampled_dataX_shortterm_movable, preprocessed_downsampled_dataY_shortterm_movable, epochs=50, only_glogen=True,
                                                batch_size=20, output_diminsion=34, use_mse=True, validation_split=0.2, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model3_pred = glocal_model3.predict(preprocessed_downsampled_dataX_shortterm_movable)
glocal_model3_npss = Metrics().NPSS(preprocessed_downsampled_dataY_shortterm_movable, glocal_model3_pred)
glocal_model3_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_shortterm_movable, glocal_model3_pred, number_of_joints=17)

print('Model 3:\nNPSS: {}\nMPJPE: {}'.format(glocal_model3_npss, glocal_model3_mpjpe))

##### Running Experiment with ONLY Glogen model and Joint Loss

In [ ]:
history_model4, glocal_model4 = run_experiment(preprocessed_downsampled_dataX_shortterm_movable, preprocessed_downsampled_dataY_shortterm_movable, epochs=50, only_glogen=True,
                                                batch_size=20, output_diminsion=34, validation_split=0.2, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model4_pred = glocal_model4.predict(preprocessed_downsampled_dataX_shortterm_movable)
glocal_model4_npss = Metrics().NPSS(preprocessed_downsampled_dataY_shortterm_movable, glocal_model4_pred)
glocal_model4_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_shortterm_movable, glocal_model4_pred, number_of_joints=17)

print('Model 4:\nNPSS: {}\nMPJPE: {}'.format(glocal_model4_npss, glocal_model4_mpjpe))

##### Running Experiment with ONLY Glogen model, MSE Loss and Self Attention

In [ ]:
history_model5, glocal_model5 = run_experiment(preprocessed_downsampled_dataX_shortterm_movable, preprocessed_downsampled_dataY_shortterm_movable, epochs=50, only_glogen=True,
                                                batch_size=20, output_diminsion=34, use_mse=True, validation_split=0.2, self_attention=True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model5_pred = glocal_model5.predict(preprocessed_downsampled_dataX_shortterm_movable)
glocal_model5_npss = Metrics().NPSS(preprocessed_downsampled_dataY_shortterm_movable, glocal_model5_pred)
glocal_model5_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_shortterm_movable, glocal_model5_pred, number_of_joints=17)

print('Model 5:\nNPSS: {}\nMPJPE: {}'.format(glocal_model5_npss, glocal_model5_mpjpe))

##### Running Experiment with ONLY Glogen model, Joint Loss and Cross Attention

In [ ]:
history_model6, glocal_model6 = run_experiment(preprocessed_downsampled_dataX_shortterm_movable, preprocessed_downsampled_dataY_shortterm_movable, epochs=50, only_glogen=True,
                                                batch_size=20, output_diminsion=34, validation_split=0.2, cross_attention=True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model6_pred = glocal_model6.predict(preprocessed_downsampled_dataX_shortterm_movable)
glocal_model6_npss = Metrics().NPSS(preprocessed_downsampled_dataY_shortterm_movable, glocal_model6_pred)
glocal_model6_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_shortterm_movable, glocal_model6_pred, number_of_joints=17)

print('Model 6:\nNPSS: {}\nMPJPE: {}'.format(glocal_model6_npss, glocal_model6_mpjpe))

##### Experimentations with Self Attention Mechanism and MSE Loss

In [ ]:
history_model7, glocal_model7 = run_experiment(preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable, batch_size=20, dropout=0.0, epochs=50, interpolate_frames=2,
                                               use_mse=True, output_diminsion=34, validation_split=0.2, self_attention = True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model7_pred = glocal_model7.predict(preprocessed_downsampled_dataX_movable)
glocal_model7_npss = Metrics().NPSS(preprocessed_downsampled_dataY_movable, glocal_model7_pred)
glocal_model7_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_movable, glocal_model7_pred, number_of_joints=17)

print('Model 7:\nNPSS: {}\nMPJPE: {}'.format(glocal_model7_npss, glocal_model7_mpjpe))

##### Experimentations with Self Attention Mechanism and Joint Loss

In [ ]:
history_model8, glocal_model8 = run_experiment(preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable, batch_size=20, interpolate_frames=2,
                                                 dropout=0.0, epochs=50, output_diminsion=34, validation_split=0.2, self_attention = True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model8_pred = glocal_model8.predict(preprocessed_downsampled_dataX_movable)
glocal_model8_npss = Metrics().NPSS(preprocessed_downsampled_dataY_movable, glocal_model8_pred)
glocal_model8_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_movable, glocal_model8_pred, number_of_joints=17)

print('Model 8:\nNPSS: {}\nMPJPE: {}'.format(glocal_model8_npss, glocal_model8_mpjpe))

##### Experimentations with Cross Attention Mechanism and MSE Loss

In [ ]:
history_model9, glocal_model9 = run_experiment(preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable, batch_size=20, dropout=0.0, epochs=50, interpolate_frames=2, 
                                               use_mse=True, output_diminsion=34, validation_split=0.2, cross_attention = True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model9_pred = glocal_model9.predict(preprocessed_downsampled_dataX_movable)
glocal_model9_npss = Metrics().NPSS(preprocessed_downsampled_dataY_movable, glocal_model9_pred)
glocal_model9_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_movable, glocal_model9_pred, number_of_joints=17)

print('Model 9:\nNPSS: {}\nMPJPE: {}'.format(glocal_model9_npss, glocal_model9_mpjpe))

##### Experimentations with Cross Attention Mechanism and Joint Loss

In [ ]:
history_model10, glocal_model10 = run_experiment(preprocessed_downsampled_dataX_movable, preprocessed_downsampled_dataY_movable, batch_size=20, dropout=0.0, epochs=50, interpolate_frames=2,
                                                 output_diminsion=34, validation_split=0.2, cross_attention = True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [ ]:
glocal_model10_pred = glocal_model10.predict(preprocessed_downsampled_dataX_movable)
glocal_model10_npss = Metrics().NPSS(preprocessed_downsampled_dataY_movable, glocal_model10_pred)
glocal_model10_mpjpe = Metrics().MPJPE2(preprocessed_downsampled_dataY_movable, glocal_model10_pred, number_of_joints=17)

print('Model 10:\nNPSS: {}\nMPJPE: {}'.format(glocal_model10_npss, glocal_model10_mpjpe))

## Loss Graphs

In [ ]:
from matplotlib.lines import lineStyles


fig = plt.figure(figsize=(20,10))

plt.plot(history_simple_RNN6.history['loss'][:10], linestyle='--', label='GRU with Linear Activation')

plt.plot(history_model1.history['loss'][:10], label='Glogen with All Joints Without Preprocessing')
plt.plot(history_model2.history['loss'][:10], label='GLogen with All Joints With Preprocessing')
plt.plot(history_model3.history['loss'][:10], label='Glogen with Movable Joints With Preprocessing')

plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()

plt.show()

## Visualization

In [ ]:
visualize_frames(glocal_model5_pred[4000], dynamic_joints_only=True)

In [ ]:
visualize_frames(preprocessed_downsampled_dataY_movable[4000], dynamic_joints_only=True)

# References

[1] https://github.com/una-dinosauria/3d-pose-baseline/blob/master/src/data_utils.py

[2] Gopalakrishnan, Anand, et al. "A neural temporal model for human motion prediction." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2019

[3] https://github.com/cr7anand/neural_temporal_models